### 11.7 특잇값 분해

In [1]:
import torch

M = torch.randn((4,3))
U, S, V = torch.svd(M)
compose_M = torch.mm(torch.mm(U, torch.diag(S)), V.t())

print(M)
print(compose_M)

tensor([[ 0.5146,  1.3459,  0.4833],
        [ 0.5243,  0.4914,  1.2455],
        [ 1.7644, -0.2290, -0.3722],
        [ 0.7255,  0.7523,  0.9117]])
tensor([[ 0.5146,  1.3459,  0.4833],
        [ 0.5243,  0.4914,  1.2455],
        [ 1.7644, -0.2290, -0.3722],
        [ 0.7255,  0.7523,  0.9117]])


### 11.8 특잇값 분해를 이용한 저계수 분해

In [2]:
import torch

M = torch.randn((4,3))
k = 2

Uk, sk, Vk = torch.svd_lowrank(M, q=k)
approximate_M = torch.mm(torch.mm(Uk, torch.diag(sk)), Vk.t())
print(M)
print(approximate_M)

tensor([[-0.1725, -0.0918,  1.1995],
        [-1.1786,  1.1210, -2.0153],
        [ 1.2107, -2.0967, -0.5269],
        [-1.3934,  0.5695, -2.2502]])
tensor([[ 0.1904,  0.1618,  1.1006],
        [-1.2780,  1.0516, -1.9882],
        [ 1.2757, -2.0512, -0.5447],
        [-1.1509,  0.7389, -2.3163]])


 ### 11.9 VGG-16 합성곱 계층 경량화

In [7]:
import torch
import tensorly as tl
from torch import nn
from torchvision import models
from tensorly import decomposition

tl.set_backend("pytorch")


def cp_decomposition(layer, rank):
    weights, factors = decomposition.parafac(
        tensor=layer.weight.data,
        rank=rank,
        init="random",
        normalize_factors=False
    )
    last, first, vertical, horizontal = factors

    pointwise_s_to_r_layer = nn.Conv2d(
        first.shape[0],
        first.shape[1],
        kernel_size=1,
        stride=1,
        padding=0,
        dilation=layer.dilation,
        bias=False,
    )
    depthwise_vertical_layer = nn.Conv2d(
        vertical.shape[1],
        vertical.shape[1],
        kernel_size=(vertical.shape[0], 1),
        stride=1,
        padding=(layer.padding[0], 0),
        dilation=layer.dilation,
        groups=vertical.shape[1],
        bias=False,
    )
    depthwise_horizontal_layer = nn.Conv2d(
        horizontal.shape[1],
        horizontal.shape[1],
        kernel_size=(1, horizontal.shape[0]),
        stride=layer.stride,
        padding=(0, layer.padding[0]),
        dilation=layer.dilation,
        groups=horizontal.shape[1],
        bias=False,
    )
    pointwise_r_to_t_layer = nn.Conv2d(
        last.shape[1],
        last.shape[0],
        kernel_size=1,
        stride=1,
        padding=0,
        dilation=layer.dilation,
        bias=True,
    )
    pointwise_r_to_t_layer.bias.data = layer.bias.data

    depthwise_horizontal_layer.weight.data = (
        torch.transpose(horizontal, 1, 0).unsqueeze(1).unsqueeze(1)
    )
    depthwise_vertical_layer.weight.data = (
        torch.transpose(vertical, 1, 0).unsqueeze(1).unsqueeze(-1)
    )
    pointwise_s_to_r_layer.weight.data = (
        torch.transpose(first, 1, 0).unsqueeze(-1).unsqueeze(-1)
    )
    pointwise_r_to_t_layer.weight.data = last.unsqueeze(-1).unsqueeze(-1)

    new_layers = [
        pointwise_s_to_r_layer,
        depthwise_vertical_layer,
        depthwise_horizontal_layer,
        pointwise_r_to_t_layer,
    ]
    return nn.Sequential(*new_layers)


model = models.vgg16(num_classes=2)
model.load_state_dict(torch.load("../models/VGG16.pt"))
model.eval()

layer = model.features[0]
layer_cp_decomposed = cp_decomposition(layer, rank=16)

print("CP 분해 전 가중치 수:", sum(param.numel() for param in layer.parameters()))
print("CP 분해 후 가중치 수:", sum(param.numel() for param in layer_cp_decomposed.parameters()))

CP 분해 전 가중치 수: 1792
CP 분해 후 가중치 수: 1232


### 11.10 VGG-16 모델 경량화

In [8]:
import copy


decomposed_model = copy.deepcopy(model)
for idx, module in enumerate(decomposed_model.features):
    if isinstance(module, nn.Conv2d):
        rank = max(module.weight.data.numpy().shape) // 3
        decomposed_model.features[idx] = cp_decomposition(module, rank)

print("CP 분해 전 가중치 수 :", sum(param.numel() for param in model.parameters()))
print("CP 분해 후 가중치 수 :", sum(param.numel() for param in decomposed_model.parameters()))

CP 분해 전 가중치 수 : 134268738
CP 분해 후 가중치 수 : 120710231


### 11.11 ONNX 형식변환

In [9]:
import torch
from torch import onnx
from torchvision import models


model = models.vgg16(num_classes=2)
model.load_state_dict(torch.load("../models/VGG16.pt"))
model.eval()

dummy_input = torch.randn(1, 3, 224, 224)
onnx.export(model=model, args=dummy_input, f="../models/VGG16.onnx")

### 11.12 ONNX 런타임 실행

In [10]:
import time
import torch
import onnxruntime as ort
from PIL import Image
from torchvision import models
from torchvision import transforms


def to_numpy(tensor):
    return tensor.detach().cpu().numpy()


image = Image.open("../datasets/images/cat.jpg")
transform = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.48235, 0.45882, 0.40784],
            std=[0.229, 0.224, 0.225],
        ),
    ]
)
input = transform(image).unsqueeze(0)

model = models.vgg16(num_classes=2)
model.load_state_dict(torch.load("../models/VGG16.pt"))
model.eval()

with torch.no_grad():
    start_time = time.time()
    output = model(input)
    end_time = time.time()
    print("파이토치:")
    print(output)
    print(end_time - start_time)


ort_session = ort.InferenceSession("../models/VGG16.onnx")

start_time = time.time()
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(input)}
ort_outs = ort_session.run(output_names=None, input_feed=ort_inputs)
end_time = time.time()
print("ONNX:")
print(ort_outs)
print(end_time - start_time)

파이토치:
tensor([[0.0109, 0.0583]])
0.10271596908569336
ONNX:
[array([[0.01094017, 0.05831044]], dtype=float32)]
0.1079719066619873


### 11.13 BERT 모델 클래스 선언

In [1]:
# app_flask.py
import torch
from torch.nn import functional as F
from transformers import BertTokenizer, BertForSequenceClassification


class BertModel:
    device = "cuda" if torch.cuda.is_available() else "cpu"

    @classmethod
    def load_model(cls, weight_path):
        cls.tokenizer = BertTokenizer.from_pretrained(
            pretrained_model_name_or_path="bert-base-multilingual-cased",
            do_lower_case=False,
        )
        cls.model = BertForSequenceClassification.from_pretrained(
            pretrained_model_name_or_path="bert-base-multilingual-cased",
            num_labels=2
        ).to(cls.device)
        cls.model.load_state_dict(torch.load(weight_path, map_location=cls.device))
        cls.model.eval()

    @classmethod
    def preprocessing(cls, data):
        input_data = cls.tokenizer(
            text=data,
            padding="longest",
            truncation=True,
            return_tensors="pt"
        ).to(cls.device)
        return input_data

    @classmethod
    @torch.no_grad()
    def predict(cls, input):
        input_data = cls.preprocessing(input)
        outputs = cls.model(**input_data).logits
        probs = F.softmax(outputs, dim=-1)

        index = int(probs[0].argmax(axis=-1))
        label = "긍정" if index == 1 else "부정"
        score = float(probs[0][index])

        return {
            "label": label,
            "score": score
        }

/Users/chohi/project/ai/DeepLearningDeepen/DeepLearningAdv/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/chohi/project/ai/DeepLearningDeepen/DeepLearningAdv/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# app_flask.py
import json
from flask import Flask, request, Response


app = Flask(__name__)


@app.route("/predict", methods=["POST"])
def inference():
    data = request.get_json()
    text = data["text"]

    try:
        return Response(
            response=json.dumps(BertModel.predict(text), ensure_ascii=False),
            status=200,
            mimetype="application/json",
        )

    except Exception as e:
        return Response(
            response=json.dumps({"error": str(e)}, ensure_ascii=False),
            status=500,
            mimetype="application/json",
        )


if __name__ == "__main__":
    BertModel.load_model(weight_path="../models/BertForSequenceClassification.pt")
    app.run(host="0.0.0.0", port=8000)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://192.168.31.231:8000
Press CTRL+C to quit


### 11.16 모델 추론 요청

In [15]:
import json
import requests


url = "http://127.0.0.1:8000/predict"
headers = {"content-type": "application/json"}

response = requests.post(
    url=url,
    headers=headers,
    data=json.dumps({"text": "정말 재미 있어요!"})
)

print(response.status_code)
print(response.json())

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=8000): Max retries exceeded with url: /predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x300c51340>: Failed to establish a new connection: [Errno 61] Connection refused'))

### 11.18 fast API

In [22]:
# app_fastapi.py
import io
import torch
import base64
from PIL import Image
from torch.nn import functional as F
from torchvision import models, transforms


class VGG16Model:
    def __init__(self, weight_path):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.transform = transforms.Compose(
            [
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(
                    mean=[0.48235, 0.45882, 0.40784],
                    std=[0.229, 0.224, 0.225]
                )
            ]
        )
        self.model = models.vgg16(num_classes=2).to(self.device)
        self.model.load_state_dict(torch.load(weight_path, map_location=self.device))
        self.model.eval()

    def preprocessing(self, data):
        decode = base64.b64decode(data)
        bytes = io.BytesIO(decode)
        image = Image.open(bytes)
        input_data = self.transform(image).to(self.device)
        return input_data

    @torch.no_grad()
    def predict(self, input):
        input_data = self.preprocessing(input)
        outputs = self.model(input_data.unsqueeze(0))
        probs = F.softmax(outputs, dim=-1)

        index = int(probs[0].argmax(axis=-1))
        label = "개" if index == 1 else "고양이"
        score = float(probs[0][index])

        return {
            "label": label,
            "score": score
        }

In [30]:
# app_fastapi.py
import uvicorn
from pydantic import BaseModel
from fastapi import FastAPI, Depends, HTTPException


app = FastAPI()
vgg = VGG16Model(weight_path="../models/VGG16.pt")


class Item(BaseModel):
    base64: str


def get_model():
    return vgg

@app.get("/")
async def read_root():
    return {"Hello": "World"}


@app.post("/predict")
async def inference(item: Item, model: VGG16Model = Depends(get_model)):
    try:
        return model.predict(item.base64)

    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))


if __name__ == "__main__":
    uvicorn.run(app="app_fastapi:app", host="0.0.0.0", port=8000, workers=2)

ERROR:    [Errno 48] Address already in use


AttributeError: 'tuple' object has no attribute 'tb_frame'

### 11.20 추론모델 요청

In [29]:
import io
import json
import base64
import requests
from PIL import Image


url = "http://127.0.0.1:8001/predict"
headers = {"content-type": "application/json"}

image = Image.open("../datasets/images/dog.jpg")
with io.BytesIO() as buffer:
    image.save(buffer, format="JPEG")
    buffer.seek(0)
    bytes = buffer.read()
string = base64.b64encode(bytes).decode("utf-8")

response = requests.post(
    url=url,
    headers=headers,
    data=json.dumps({"base64": string})
)

print(response.status_code)
print(response.json())

response = requests.get(
    url="http://127.0.0.1:8001/",
    headers=headers,
    data=json.dumps({"base64": string})
)

print(response.status_code)
print(response.json())

200
{'label': '개', 'score': 0.5182952284812927}
200
{'Hello': 'World'}


### 11.34 애플리케이션 구성

In [1]:
# demo.py
import pandas as pd
import streamlit as st


st.set_page_config(
    page_title="데모 애플리케이션",
    page_icon=":shark:",
    layout="wide"
)

df = pd.read_csv("../datasets/non_linear.csv")

st.header(body="Demo Application")
st.subheader(body="non_linear.csv")

x = st.sidebar.selectbox(label="X 축", options=df.columns, index=0)
y = st.sidebar.selectbox(label="Y 축", options=df.columns, index=1)

col1, col2 = st.columns(2)
with col1:
    st.dataframe(data=df, height=500, use_container_width=True)
with col2:
    st.line_chart(data=df, x=x, y=y, height=500)

2025-03-29 17:50:53.634 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 17:50:53.639 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 17:50:53.737 
  command:

    streamlit run /Users/chohi/project/ai/DeepLearningDeepen/DeepLearningAdv/.venv/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-03-29 17:50:53.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 17:50:53.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 17:50:53.737 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-29 17:50:53.738 Thread 'MainThread': missing Scrip